# Case - 十字彈片

In [7]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level="ERROR")
import warnings
warnings.filterwarnings("ignore")
import keras, os
import numpy as np
from random import randint
from datasets import load_hotmelt, load_hotmelt_generator
import tensorflow as tf
from sklearn.model_selection import train_test_split

from keras.applications import MobileNet, MobileNetV2,DenseNet169, NASNetLarge, InceptionResNetV2, ResNet50
from keras.applications.inception_resnet_v2 import preprocess_input
#from efficientnet import EfficientNetB0

from keras.layers import Dense,GlobalAveragePooling2D, Activation
from keras.optimizers import SGD, Adagrad
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, TensorBoard, ReduceLROnPlateau, LambdaCallback

import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw

from utils import overSampling, YOLOV3
plt.rcParams['font.size'] = 9
plt.rcParams['figure.figsize'] = (9,9)

os.environ['CUDA_VISIBLE_DEVICES']='1' 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [2]:
#data = load_hotmelt()
data = load_hotmelt_generator()

### Load YOLO Model

In [3]:
graph1 = tf.Graph()
with graph1.as_default():
    session1 = tf.Session()
    with session1.as_default():
        yolo=YOLOV3()

model/hotmelt_yolov3weight.h5 model, anchors, and classes loaded.


### Build Model

In [4]:
graph2 = tf.Graph()
with graph2.as_default():
    session2 = tf.Session()
    with session2.as_default():
        input_shape = (224, 224, 3)
        base_model=DenseNet169(weights='imagenet',include_top=False, input_shape=input_shape) 
        x=base_model.output
        x=GlobalAveragePooling2D()(x)
        preds=Dense(5)(x) #final layer with softmax activation
        preds = Activation('softmax', name='prob')(preds)    
        model=Model(inputs=base_model.input,outputs=preds)
        opt = SGD(lr= 0.01, decay=1e-04, momentum=0.85, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        for idx, layer in enumerate(model.layers):
            layer.trainable = True



In [8]:
imgAug = ImageDataGenerator(
    samplewise_center=True, samplewise_std_normalization=True,
    vertical_flip=True, horizontal_flip=True,
    width_shift_range=0.02, height_shift_range=0.02,
    zoom_range =[0.95,1.05],
)
def tr_augfun(batch_img, batch_y, batch_size):
    with graph1.as_default():
        with session1.as_default():
            roi_imgs = np.array(list(map(lambda x: yolo.predict_roi(Image.fromarray(x.astype(np.uint8))), batch_img)))
    gen = imgAug.flow(roi_imgs, batch_y, batch_size=batch_size)
    X, Y = gen.next()
    return X,Y

val_imgAug = ImageDataGenerator(
    samplewise_center=True, samplewise_std_normalization=True,
)
def val_augfun(batch_img, batch_y, batch_size):
    with graph1.as_default():
        with session1.as_default():
            roi_imgs = np.array(list(map(lambda x: yolo.predict_roi(Image.fromarray(x.astype(np.uint8))), batch_img)))
    gen = val_imgAug.flow(roi_imgs, batch_y, batch_size=batch_size)
    X, Y = gen.next()
    return X,Y

batch=19
early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=100, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
#checkpointer=ModelCheckpoint(monitor='loss',filepath='model/ap5_tmp.h5', verbose=1, save_best_only=True, save_weights_only=True)
callbacks = [reduce_lr, early_stop]
    
print('start training...')
#model training
with graph2.as_default():
    with session2.as_default():
        history = model.fit_generator(data.dataGenerator(11, tr_augfun, dtype='tr'), steps_per_epoch=100, validation_steps=11, epochs=50, validation_data=data.dataGenerator(11, val_augfun, dtype='val'), callbacks=callbacks) 
#history = model.fit_generator(data.dataGenerator(10, augfun), steps_per_epoch=X_tr.shape[0]//batch, validation_steps=X_val.shape[0]//batch, epochs=50, validation_data=dgen_val, callbacks=callbacks) 



start training...
Epoch 1/50
184
2375
  4/100 [>.............................] - ETA: 44:57 - loss: 1.6175 - acc: 0.4091

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(6,6))
plt.subplot(211)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.subplot(212)
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.legend()
plt.ylim((0, 1.1))
plt.show()

### Check Test Data Peroformance

In [ ]:
predGen = ImageDataGenerator(
    samplewise_center=True, samplewise_std_normalization=True,
)

ret = model.evaluate_generator(predGen.flow(X_tr, y_tr, shuffle=False), steps=X_tr.shape[0], verbose=1)
print('tr,', 'loss:', ret[0], 'acc:',ret[1])

ret = model.evaluate_generator(predGen.flow(X_val, y_val,shuffle=False), steps=X_val.shape[0], verbose=1)
print('val', 'loss:', ret[0], 'acc:',ret[1])

ret = model.evaluate_generator(predGen.flow(X_ts, y_ts,shuffle=False), steps=X_ts.shape[0], verbose=1)
print('ts',  'loss:', ret[0], 'acc:',ret[1])

### Tensorlite model

In [ ]:
tflite_model_file = 'tfmodel/hotmelt.tflite'
modelname='model/hotmelt.h5'
model.save(modelname)
converter = tf.lite.TFLiteConverter.from_keras_model_file(modelname, input_arrays=['input_7'], input_shapes={'input_7':[None,input_shape[0],input_shape[1],input_shape[2]]})
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)   

  

In [ ]:
tflite_model_file = 'tfmodel/hotmelt.tflite'
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]      
nb_test_samples = X_ts.shape[0]
nb_rows, nb_cols = 5, 5
    
preds=[]
idxs=[]
for k in range(nb_rows * nb_cols):
    i = randint(0, nb_test_samples - 1)
    idxs.append(i)
    interpreter.set_tensor(input_index, np.expand_dims(preprocess_input(X_ts[i].copy()), axis=0))
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    preds.append(pred.flatten())

plt.figure(figsize=(10,10))
for k in range(nb_rows * nb_cols):
    plt.subplot(nb_rows, nb_cols, k+1)
    plt.imshow(X_ts[idxs[k]].astype(np.uint8))
    r = 'OK' if np.argmax(preds[k]) == np.argmax(y_ts[idxs[k]]) else 'NG'
    plt.title(f'p:{np.argmax(preds[k])}, a:{np.argmax(y_ts[idxs[k]])}, {r}')
    plt.axis('off')    
plt.tight_layout()
plt.show()
    
